In [1]:
import cv2

In [2]:
def extract_frames(video_path):
  frames = []
  video_capture = cv2.VideoCapture(video_path)
  while(True):
    ret, frame = video_capture.read()
    if ret == False:
      break
    frames.append(frame)
  video_capture.release()
  return frames

def save_video(output_video_frames, path):
  height, width, _ = output_video_frames[0].shape
  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  out = cv2.VideoWriter(path, fourcc, 30, (width, height))
  for frame in output_video_frames:
      out.write(frame)
  out.release()

In [3]:
#Returns frames of the input video
frames = extract_frames("point.mp4")

#Detecting players

In [ ]:
!pip install ultralytics
from ultralytics import YOLO

In [ ]:
model = YOLO("models/yolov8x.pt")
player_detections = [] #List of dictionaries for each frame
for frame in frames:
  results = model.track(frame, persist = True)[0]
  player_dict = {} #Key:player_id value:coordinates of a box
  for box in results.boxes:
    id = int(box.id.tolist()[0])
    result = box.xyxy.tolist()[0]
    class_name = results.names[box.cls.tolist()[0]]
    if class_name == "person":
      player_dict[id] = result
  player_detections.append(player_dict)

In [8]:
#Drawing bounding boxes
output_frames=[]
for frame, player_dict in zip(frames, player_detections):
  for id, box in player_dict.items():
    x1, y1, x2, y2 = box
    color = (0, 255, 0)
    thickness = 2
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
    cv2.putText(frame, f"Player: {id}", (int(box[0]), int(box[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
  output_frames.append(frame)

In [9]:
#Saves the video
#Input is video frames and path
save_video(output_frames, "output_point.avi")